<a href="https://colab.research.google.com/github/dinggga/super-broccoli/blob/main/220523hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **9.8 파이썬 추천 시스템 패키지 - Surprise**

**Surprise 패키지 소개**
- 파이썬 기반 추천 시스템 구축을 위한 전용 패키지. 사이킷런과 유사한 API와 프레임워크 제공
- 장점
  - 다양한 추천 알고리즘 ex_ 사용자/아이템 기반 최근접 이웃 협업 필터링, SVD, SVD++, NMF 기반 잠재 요인 협업 필터링
  - 사이킷런의 핵심 API와 유사한 API명 (fit(), predict(), train_test_split(), cross_validate(), GridSearchCV 클래스 등)

**Surprise를 이용한 추천 시스템 구축**



In [2]:
!pip3 install surprise

     |████████████████████████████████| 11.8 MB 13.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633704 sha256=6106c5aba065b9b8fc2420da7f399fc96cb541049018d7e56cb8a3acaeafd801
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

Surprise에서 데이터 로딩은 Dataset 클래스를 이용해서만 가능

주요 데이터가 로우(Row) 레벨 형태로 돼있는 포맷의 데이터만 처리

In [4]:
data = Dataset.load_builtin('ml-100k') 
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


'ml-100k' 데이터 세트: 과거 버전의 데이터 세트. 분리 문자가 탭(\t) 문자.

In [5]:
#SVD로 잠재 요인 협업 필터링 수행
algo = SVD()
algo.fit(trainset)

In [6]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.393000337034676, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.871996812169948, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.9903164810317038, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.772561388958829, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.667332696755239, details={'was_impossible': False})]

튜플 형태

Prediction 객체의 details 속성: 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터 남기는 데 사용. 모두 False.

In [7]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.393000337034676),
 ('882', '291', 3.871996812169948),
 ('535', '507', 3.9903164810317038)]

In [8]:
#predict()를 이용한 추천 예측

# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.01   {'was_impossible': False}


성능 평가: accuracy 모듈: RMSE, MSE 등의 방법으로 성능 평가 정보 제공

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9468


0.946803315880753

### **Surprise 주요 모듈 소개**

**Dataset**
- 사용자 아이디, 아이템 아이디, 평점으로 가정해 데이터 로딩하고 네번째 칼럼부터는 아예 로딩 수행 X
- Dataset.load_builtin(name='ml-100k')
- Dataset.load_from_file(file_path, reader)
- Dataset.load_from_df(df, reader)

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**



In [11]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ESAA_OB/220516M/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('/content/drive/MyDrive/ESAA_OB/220516M/ratings_noh.csv', index=False, header=False)

In [12]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/drive/MyDrive/ESAA_OB/220516M/ratings_noh.csv',reader=reader)

Reader 클래스의 주요 생성 파라미터
- line_format(string)
- sep(char)
- rating_scale(tuple, optional)

In [13]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**

In [15]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/ESAA_OB/220516M/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

### **Surprise 추천 알고리즘 클래스**

- SVD
- KNNBasic
- BaselineOnly 등

SVD 클래스의 입력 파라미터
- n_factors
- n_epochs
- biased(bool)

### **베이스라인 평점(Baseline Rating)**

- 개인의 성향 반영해 아이템 평가에 편향성(bias) 요소 반영하여 평점 부과
- 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수

### **교차검증과 하이퍼 파라미터 튜닝**

- cross_validate(), GridSearchCV 클래스 제공
- cross_validate() 함수

In [16]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv('/content/drive/MyDrive/ESAA_OB/220516M/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True) 

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8979  0.8929  0.8931  0.8996  0.9027  0.8972  0.0038  
MAE (testset)     0.6905  0.6859  0.6879  0.6921  0.6975  0.6908  0.0040  
Fit time          5.35    5.34    6.12    5.06    5.22    5.42    0.37    
Test time         0.31    0.14    0.15    0.25    0.14    0.20    0.07    


{'fit_time': (5.349052667617798,
  5.338528871536255,
  6.124296426773071,
  5.060783624649048,
  5.223031759262085),
 'test_mae': array([0.69051743, 0.6859414 , 0.68786975, 0.6920887 , 0.69751   ]),
 'test_rmse': array([0.89786983, 0.89291954, 0.89310873, 0.89959696, 0.90265572]),
 'test_time': (0.3142073154449463,
  0.13817143440246582,
  0.14710426330566406,
  0.24591851234436035,
  0.13989663124084473)}

- n_epochs: SGD의 반복 횟수 지정
- n_factors: 잠재 요인 K의 크기 지정

In [17]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8995872989621382
{'n_epochs': 20, 'n_factors': 50}


### **Surprise를 이용한 개인화 영화 추천 시스템 구축**



In [19]:
# 아래 코드는 train_test_split( )으로 분리되지 않는 Dataset에 fit( )을 호출하여 오류를 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: ignored

In [ ]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/ESAA_OB/220516M/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

과제 다 못했습니다..ㅠㅠ